In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  1 18:26:35 2021

@author: rahul
"""
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

import os

os.chdir("D:\\Learning\\Customer Dataloder pytorch youtube\\Machine-Learning-Collection-master\\ML\Pytorch\\Basics\\custom_dataset")




In [2]:
from random import randrange
print(randrange(10))

7


In [3]:
rand = randrange(10)
if rand%2==0:
    print (rand)

In [4]:
# https://stackoverflow.com/questions/51585298/data-loading-with-variable-batch-size
    
# https://stackoverflow.com/questions/55041080/how-does-pytorch-dataloader-handle-variable-size-data
    
# https://github.com/pytorch/vision/issues/555

# http://seba1511.net/tutorials/beginner/data_loading_tutorial.html

In [5]:
# class CatsAndDogsDataset(Dataset):
#     def __init__(self, csv_file, root_dir, transform=None):
#         self.annotations = pd.read_csv(csv_file)
#         self.root_dir = root_dir
#         self.transform = transform
        

#     def __len__(self):
#         return len(self.annotations)

#     def __getitem__(self, index):
#         images = []
#         y_labels = []
#         img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
#         image = io.imread(img_path)
#         y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

#         if self.transform:
#             image = self.transform(image)
            
#         images.append(image)

#         y_labels.append(y_label)
        
#         rand = randrange(10)
#         if rand%2==0:
#             y_labels.append(y_label)
#             images.append(image)
        

#         return [images, y_labels]




In [11]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        images = []
        y_labels = []
        final = []
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)
            
        images.append(image)

        y_labels.append(y_label)
        
        rand = randrange(10)
        if rand%2==0:
            y_labels.append(y_label)
            images.append(image)
        
        final.append([images, y_labels])
        return final



In [12]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# Load Data
dataset = CatsAndDogsDataset(
    csv_file="cats_dogs.csv",
    root_dir="cats_dogs_resized",
    transform=transforms.ToTensor(),
)



In [13]:
def collate_fn(batch):
    return CatsAndDogsDataset(batch)

In [14]:
# Dataset is actually a lot larger ~25k images, just took out 10 pictures
# to upload to Github. It's enough to understand the structure and scale
# if you got more images.
train_set, test_set = torch.utils.data.random_split(dataset, [5, 5])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True,collate_fn=collate_fn)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [22]:
next(iter(train_loader))

TypeError: __init__() missing 1 required positional argument: 'root_dir'

In [17]:
for batch_ndx, sample in (train_loader):
    print(sample.inp.is_pinned())
    print(sample.tgt.is_pinned())

TypeError: __init__() missing 1 required positional argument: 'root_dir'

In [16]:
# t = next(iter(train_loader))
# t.print_data()

In [ ]:
# for data, label in train_set:
#     print(type(data))
#     print(data)
#     print(type(label))
#     print(label)
#     break

In [ ]:
# dataset = list(zip(xs,ys))
# dataset[5]

In [ ]:
for data, label in train_loader:
    print(len(data))
#     print(data)
    print(len(label))
#     print(label)
#     break

In [ ]:
# train_loader.generator

In [ ]:
# for i, (expA, expB) in enumerate(train_loader):

#   print(inp.shape)
#   break

In [ ]:
import itertools

# Train Network
for epoch in range(num_epochs):
    losses = []

#     for batch_idx, (datarah, targetsrah) in enumerate(train_loader):
        # Get data to cuda if possible
    for data, targets in train_loader:        
#         data = list(itertools.chain(*datarah))
#         targets = list(itertools.chain(*targetsrah))
#         targets = list(itertools.chain(*targets))
        print("targets = ", targets)
        
        for i in range(len(targets)):
            print(targets[i])
            targets[i] = targets[i].tolist()
            
        for i in range(len(data)):
#             print(data[i])
            data[i] = data[i].tolist()
        
        print("targets = ", targets)
        print (type(targets))

        data = torch.tensor(data)
        targets = torch.tensor(targets)
        
        print ("size of the each data in epoches = ", len(data))
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")



In [ ]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            
            
            for i in range(len(y)):
                y[i] = y[i].tolist()
            
            for i in range(len(x)):
                x[i] = x[i].tolist()
                
            x = torch.tensor(x)
            y = torch.tensor(y)
            x = x.to(device=device)
            y = y.to(device=device)

#             scores = model(x)
#             _, predictions = scores.max(1)
#             num_correct += (predictions == y).sum()
#             num_samples += predictions.size(0)

#         print(
#             f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
#         )

#     model.train()




In [ ]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)
